In [1]:
import pandas as pd

# Load input data

In [2]:
cod_post_df_columns = pd.read_csv('input_data\codigos_postais.csv').columns

# Another data source:
cod_post_original_df = pd.read_csv('input_data\\todos_cp\\todos_cp.txt',
                                   names=cod_post_df_columns,
                                   delimiter=';',
                                   encoding='LATIN-1')
concelhos_df = pd.read_csv('input_data\\todos_cp\\concelhos.txt',
                                   names=['cod_distrito', 'cod_concelho', 'concelho'],
                                   delimiter=';',
                                   encoding='LATIN-1')
distritos_df = pd.read_csv('input_data\\todos_cp\\distritos.txt',
                                   names=['cod_distrito', 'distrito'],
                                   delimiter=';',
                                   encoding='LATIN-1')

cod_post_mapping_df = pd.read_csv('input_data\Mapeamento_Freguesias_Pre_Pos_RATF.csv', skiprows=2, encoding = "LATIN-1")

In [3]:
cod_post_original_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,cod_arteria,tipo_arteria,prep1,titulo_arteria,prep2,nome_arteria,local_arteria,troco,porta,cliente,num_cod_postal,ext_cod_postal,desig_postal
0,1,1,249,Alcafaz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,11,AGADÃO
1,1,1,250,Caselho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,12,AGADÃO
2,1,1,251,Corga da Serra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,13,AGADÃO
3,1,1,252,Foz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,14,AGADÃO
4,1,1,253,Guistola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,15,AGADÃO


In [4]:
concelhos_df.head()

,cod_distrito,cod_concelho,concelho
0,13,8,Matosinhos
1,13,9,Paços de Ferreira
2,13,10,Paredes
3,13,11,Penafiel
4,13,12,Porto


In [5]:
distritos_df.head()

,cod_distrito,distrito
0,1,Aveiro
1,2,Beja
2,3,Braga
3,4,Bragança
4,5,Castelo Branco


In [6]:
cod_post_mapping_df.head()

,Distrito/Ilha,Concelho,Freguesia Pré RATF,Alteração RATF,Freguesia Final (Pós RATF)
0,Aveiro,Águeda,Aguada de Cima,Sem alteração,Aguada de Cima
1,Aveiro,Águeda,Fermentelos,Sem alteração,Fermentelos
2,Aveiro,Águeda,Macinhata do Vouga,Sem alteração,Macinhata do Vouga
3,Aveiro,Águeda,Valongo do Vouga,Sem alteração,Valongo do Vouga
4,Aveiro,Águeda,Águeda,Agregação,União das freguesias de Águeda e Borralha


# Data Cleaning

## Check for missing values

### Original postal codes data

In [7]:
cod_post_original_df.isna().sum()

cod_distrito            0
cod_concelho            0
cod_localidade          0
nome_localidade         0
cod_arteria         30753
tipo_arteria        33307
prep1              167769
titulo_arteria     283338
prep2              322332
nome_arteria        30753
local_arteria      256318
troco              288213
porta              324401
cliente            323178
num_cod_postal          0
ext_cod_postal          0
desig_postal            0
dtype: int64

We are not interested in any of the columns with missing values, so we can safely delete them:

In [8]:
cod_post_original_df.dropna(axis=1, inplace=True)
cod_post_original_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,num_cod_postal,ext_cod_postal,desig_postal
0,1,1,249,Alcafaz,3750,11,AGADÃO
1,1,1,250,Caselho,3750,12,AGADÃO
2,1,1,251,Corga da Serra,3750,13,AGADÃO
3,1,1,252,Foz,3750,14,AGADÃO
4,1,1,253,Guistola,3750,15,AGADÃO


### Postal code mapping data

In [9]:
cod_post_mapping_df.isna().sum()

Distrito/Ilha                 0
Concelho                      0
Freguesia Pré RATF            1
Alteração RATF                0
Freguesia Final (Pós RATF)    1
dtype: int64

Here, we see that only two entries are N/A. Lets take a closer look:

In [10]:
cod_post_mapping_df[cod_post_mapping_df.isna().any(axis=1)]

,Distrito/Ilha,Concelho,Freguesia Pré RATF,Alteração RATF,Freguesia Final (Pós RATF)
283,Beja,Odemira,Bicos,Extinta - Destinos Múltiplos,NaN
2226,Lisboa,Lisboa,NaN,Nova,Parque das Nações


In one of them, the *"freguesia"* ceased to exists, so there is no name for it after the re-structuring. In the other case, a new *"freguesia"* was created, so there previously was no name for it. We can easily fix this by filling both entries with the placeholder " ":

In [11]:
cod_post_mapping_df.fillna(value=' ', inplace=True)

# Data Wrangling

We first want to match the postal code to the *Freguesia*, *Concelho* and *Distrito*. For that, we will use the two other *dataframes*:

In [12]:
distritos_df.set_index('cod_distrito', inplace=True)

In [13]:
cod_post_original_df = cod_post_original_df.join(distritos_df, on='cod_distrito', rsuffix='__')

In [14]:
concelhos_df.set_index('cod_distrito', inplace=True)
concelhos_df.set_index('cod_concelho', append=True, inplace=True)

In [15]:
cod_post_original_df = cod_post_original_df.join(concelhos_df, on=['cod_distrito', 'cod_concelho'])

In [16]:
cod_post_original_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,num_cod_postal,ext_cod_postal,desig_postal,distrito,concelho
0,1,1,249,Alcafaz,3750,11,AGADÃO,Aveiro,Águeda
1,1,1,250,Caselho,3750,12,AGADÃO,Aveiro,Águeda
2,1,1,251,Corga da Serra,3750,13,AGADÃO,Aveiro,Águeda
3,1,1,252,Foz,3750,14,AGADÃO,Aveiro,Águeda
4,1,1,253,Guistola,3750,15,AGADÃO,Aveiro,Águeda


We will also add a column to the original postal code dataframe which will hold the actual postal code:

In [17]:
two_digit_ext_idx = cod_post_original_df['ext_cod_postal'] <= 99
cod_post_original_df.loc[two_digit_ext_idx, 'cod_postal'] = cod_post_original_df['num_cod_postal'].astype('str') + '0' + \
                                                            + cod_post_original_df['ext_cod_postal'].astype('str')          # if two digit extension, add leading '0' to make it three digit

three_digit_ext_idx = cod_post_original_df['ext_cod_postal'] > 99

cod_post_original_df.loc[three_digit_ext_idx, 'cod_postal'] = cod_post_original_df['num_cod_postal'].astype('str') \
                                                            + cod_post_original_df['ext_cod_postal'].astype('str')

cod_post_original_df.head()

,cod_distrito,cod_concelho,cod_localidade,nome_localidade,num_cod_postal,ext_cod_postal,desig_postal,distrito,concelho,cod_postal
0,1,1,249,Alcafaz,3750,11,AGADÃO,Aveiro,Águeda,3750011
1,1,1,250,Caselho,3750,12,AGADÃO,Aveiro,Águeda,3750012
2,1,1,251,Corga da Serra,3750,13,AGADÃO,Aveiro,Águeda,3750013
3,1,1,252,Foz,3750,14,AGADÃO,Aveiro,Águeda,3750014
4,1,1,253,Guistola,3750,15,AGADÃO,Aveiro,Águeda,3750015


# Save cleaned data

In [18]:
cod_post_original_df.to_csv('interim_data\\cod_post_orig_clean.csv', index=False)

In [19]:
cod_post_mapping_df.to_csv('interim_data\\cod_post_map_clean.csv', index=False)